In [1]:
import requests
import pandas as pd
import os
from pathlib import Path
from etl_functions import _safe_save, extract, convert_columns_to_json, transform_products, transform_carts, create_reviews_table, load

In [2]:
links = {
    "products": "https://dummyjson.com/products",
    "carts":   "https://dummyjson.com/carts",
    "users":    "https://dummyjson.com/users"
}

In [3]:
dataframes = {}

for name, link in links.items():
    df = extract(url=link, file_path="raw_data")
    dataframes[name] = df

In [4]:
products_df_raw = dataframes["products"]
products_df_raw = convert_columns_to_json(df=products_df_raw, columns = ["tags", "reviews", "images"])
products_df_raw.head()

,id,title,description,category,price,discountPercentage,rating,stock,tags,brand,...,minimumOrderQuantity,images,thumbnail,dimensions.width,dimensions.height,dimensions.depth,meta.createdAt,meta.updatedAt,meta.barcode,meta.qrCode
0,1,Essence Mascara Lash Princess,The Essence Mascara Lash Princess is a popular...,beauty,9.99,10.48,2.56,99,"[""beauty"", ""mascara""]",Essence,...,48,"[""https://cdn.dummyjson.com/product-images/bea...",https://cdn.dummyjson.com/product-images/beaut...,15.14,13.08,22.99,2025-04-30T09:41:02.053Z,2025-04-30T09:41:02.053Z,5784719087687,https://cdn.dummyjson.com/public/qr-code.png
1,2,Eyeshadow Palette with Mirror,The Eyeshadow Palette with Mirror offers a ver...,beauty,19.99,18.19,2.86,34,"[""beauty"", ""eyeshadow""]",Glamour Beauty,...,20,"[""https://cdn.dummyjson.com/product-images/bea...",https://cdn.dummyjson.com/product-images/beaut...,9.26,22.47,27.67,2025-04-30T09:41:02.053Z,2025-04-30T09:41:02.053Z,9170275171413,https://cdn.dummyjson.com/public/qr-code.png
2,3,Powder Canister,The Powder Canister is a finely milled setting...,beauty,14.99,9.84,4.64,89,"[""beauty"", ""face powder""]",Velvet Touch,...,22,"[""https://cdn.dummyjson.com/product-images/bea...",https://cdn.dummyjson.com/product-images/beaut...,29.27,27.93,20.59,2025-04-30T09:41:02.053Z,2025-04-30T09:41:02.053Z,8418883906837,https://cdn.dummyjson.com/public/qr-code.png
3,4,Red Lipstick,The Red Lipstick is a classic and bold choice ...,beauty,12.99,12.16,4.36,91,"[""beauty"", ""lipstick""]",Chic Cosmetics,...,40,"[""https://cdn.dummyjson.com/product-images/bea...",https://cdn.dummyjson.com/product-images/beaut...,18.11,28.38,22.17,2025-04-30T09:41:02.053Z,2025-04-30T09:41:02.053Z,9467746727219,https://cdn.dummyjson.com/public/qr-code.png
4,5,Red Nail Polish,The Red Nail Polish offers a rich and glossy r...,beauty,8.99,11.44,4.32,79,"[""beauty"", ""nail polish""]",Nail Couture,...,22,"[""https://cdn.dummyjson.com/product-images/bea...",https://cdn.dummyjson.com/product-images/beaut...,21.63,16.48,29.84,2025-04-30T09:41:02.053Z,2025-04-30T09:41:02.053Z,4063010628104,https://cdn.dummyjson.com/public/qr-code.png


In [5]:
column_mapping_products_raw = {
    "id": "product_id",
    "discountPercentage": "discount_percentage",
    "warrantyInformation": "warranty_information",
    "shippingInformation": "shipping_information",
    "availabilityStatus": "availability_status",
    "returnPolicy": "return_policy",
    "minimumOrderQuantity": "min_order_quantity",
    "dimensions.width": "dimension_width",
    "dimensions.height": "dimension_height",
    "dimensions.depth": "dimension_depth",
    "meta.createdAt": "meta_created_at",
    "meta.updatedAt": "meta_updated_at",
    "meta.barcode": "meta_barcode",
    "meta.qrCode": "meta_qr_code"
}

load(df=products_df_raw, table_name="products", schema="bronze", server = "DESKTOP-QBF1FTP\SQLEXPRESS", 
     database = "Dummydb", column_mapping = column_mapping_products_raw)

✅ 194 rows loaded into bronze.products


In [6]:
carts_df_raw = dataframes["carts"]
carts_df_raw = convert_columns_to_json(df=carts_df_raw, columns = ["products"])
carts_df_raw.head()

,id,products,total,discountedTotal,userId,totalProducts,totalQuantity
0,1,"[{""id"": 168, ""title"": ""Charger SXT RWD"", ""pric...",103774.85,89686.65,33,4,15
1,2,"[{""id"": 144, ""title"": ""Cricket Helmet"", ""price...",4794.80,4288.95,142,5,20
2,3,"[{""id"": 98, ""title"": ""Rolex Submariner Watch"",...",16775.87,14144.30,108,5,13
3,4,"[{""id"": 187, ""title"": ""Golden Shoes Woman"", ""p...",456.83,443.23,87,4,17
4,5,"[{""id"": 108, ""title"": ""iPhone 12 Silicone Case...",7431.30,6676.44,134,6,20


In [7]:
column_mapping_carts_raw = {
    "id": "order_id",
    "discountedTotal": "discounted_total",
    "userId":"user_id",
    "totalProducts": "total_products",
    "totalQuantity": "total_quantity",
}

load(df=carts_df_raw, table_name="carts", schema="bronze", server = "DESKTOP-QBF1FTP\SQLEXPRESS", 
     database = "Dummydb", column_mapping=column_mapping_carts_raw)

✅ 50 rows loaded into bronze.carts


In [8]:
users_df_raw = dataframes["users"]
users_df_raw.head()

,id,firstName,lastName,maidenName,age,gender,email,phone,username,password,...,company.address.city,company.address.state,company.address.stateCode,company.address.postalCode,company.address.coordinates.lat,company.address.coordinates.lng,company.address.country,crypto.coin,crypto.wallet,crypto.network
0,1,Emily,Johnson,Smith,28,female,emily.johnson@x.dummyjson.com,+81 965-431-3024,emilys,emilyspass,...,San Francisco,Wisconsin,WI,37657,71.814525,-161.150263,United States,Bitcoin,0xb9fc2fe63b2a6c003f1c324c3bfa53259162181a,Ethereum (ERC20)
1,2,Michael,Williams,,35,male,michael.williams@x.dummyjson.com,+49 258-627-6644,michaelw,michaelwpass,...,Los Angeles,New Hampshire,NH,73442,79.098326,-119.624845,United States,Bitcoin,0xb9fc2fe63b2a6c003f1c324c3bfa53259162181a,Ethereum (ERC20)
2,3,Sophia,Brown,,42,female,sophia.brown@x.dummyjson.com,+81 210-652-2785,sophiab,sophiabpass,...,Dallas,Nevada,NV,88511,20.086743,-34.577107,United States,Bitcoin,0xb9fc2fe63b2a6c003f1c324c3bfa53259162181a,Ethereum (ERC20)
3,4,James,Davis,,45,male,james.davis@x.dummyjson.com,+49 614-958-9364,jamesd,jamesdpass,...,Fort Worth,Pennsylvania,PA,27768,54.911930,-79.498328,United States,Bitcoin,0xb9fc2fe63b2a6c003f1c324c3bfa53259162181a,Ethereum (ERC20)
4,5,Emma,Miller,Johnson,30,female,emma.miller@x.dummyjson.com,+91 759-776-1614,emmaj,emmajpass,...,San Antonio,Idaho,ID,21965,44.346545,-26.944701,United States,Bitcoin,0xb9fc2fe63b2a6c003f1c324c3bfa53259162181a,Ethereum (ERC20)


In [9]:
column_mapping_users_raw= {
    "id": "user_id",
    "firstName": "first_name",
    "lastName": "last_name",
    "maidenName": "maiden_name",
    "birthDate": "birthdate",
    "bloodGroup": "blood_group",
    "eyeColor": "eye_color",
    "macAddress": "mac_address",
    "userAgent": "user_agent",
    "hair.color": "hair_color",
    "hair.type": "hair_type",
    "address.address": "address_street",
    "address.city":"address_city",
    "address.state":"address_state",
    "address.stateCode": "address_state_code",
    "address.postalCode": "address_postal_code",
    "address.coordinates.lat": "address_coordinates_lat",
    "address.coordinates.lng": "address_coordinates_lng",
    "address.country":"address_country",
    "bank.cardExpire": "bank_card_expire",
    "bank.cardNumber": "bank_card_number",
    "bank.cardType": "bank_card_type",
    "bank.currency": "bank_currency",
    "bank.iban": "bank_iban",
    "company.department": "company_deparment",
    "company.name": "company_name",
    "company.title": "company_title",
    "company.address.address": "company_address_street",
    "company.address.city": "company_address_city",
    "company.address.state": "company_address_state",
    "company.address.stateCode": "company_address_state_code",
    "company.address.postalCode": "company_address_postal_code",
    "company.address.coordinates.lat": "company_address_coordinates_lat",
    "company.address.coordinates.lng": "company_address_coordinates_lng",
    "company.address.country": "company_address_country",
    "crypto.coin": "crypto_coin",
    "crypto.wallet": "crypto_wallet",
    "crypto.network": "crypto_network"
}


load(df=users_df_raw, table_name="users", schema="bronze", server = "DESKTOP-QBF1FTP\SQLEXPRESS", 
     database = "Dummydb", column_mapping=column_mapping_users_raw)

✅ 208 rows loaded into bronze.users


In [10]:
products_df_transformed = transform_products(products_df_raw, out_dir="processed_data")
products_df_transformed.head()

,id,title,description,category,subcategory,price,discountPercentage,rating,stock,brand,...,image_5,image_6,thumbnail,dimensions.width,dimensions.height,dimensions.depth,meta.createdAt,meta.updatedAt,meta.barcode,meta.qrCode
0,1,Essence Mascara Lash Princess,The Essence Mascara Lash Princess is a popular...,beauty,mascara,9.99,10.48,2.56,99,Essence,...,None,None,https://cdn.dummyjson.com/product-images/beaut...,15.14,13.08,22.99,2025-04-30T09:41:02.053Z,2025-04-30T09:41:02.053Z,5784719087687,https://cdn.dummyjson.com/public/qr-code.png
1,2,Eyeshadow Palette with Mirror,The Eyeshadow Palette with Mirror offers a ver...,beauty,eyeshadow,19.99,18.19,2.86,34,Glamour Beauty,...,None,None,https://cdn.dummyjson.com/product-images/beaut...,9.26,22.47,27.67,2025-04-30T09:41:02.053Z,2025-04-30T09:41:02.053Z,9170275171413,https://cdn.dummyjson.com/public/qr-code.png
2,3,Powder Canister,The Powder Canister is a finely milled setting...,beauty,face powder,14.99,9.84,4.64,89,Velvet Touch,...,None,None,https://cdn.dummyjson.com/product-images/beaut...,29.27,27.93,20.59,2025-04-30T09:41:02.053Z,2025-04-30T09:41:02.053Z,8418883906837,https://cdn.dummyjson.com/public/qr-code.png
3,4,Red Lipstick,The Red Lipstick is a classic and bold choice ...,beauty,lipstick,12.99,12.16,4.36,91,Chic Cosmetics,...,None,None,https://cdn.dummyjson.com/product-images/beaut...,18.11,28.38,22.17,2025-04-30T09:41:02.053Z,2025-04-30T09:41:02.053Z,9467746727219,https://cdn.dummyjson.com/public/qr-code.png
4,5,Red Nail Polish,The Red Nail Polish offers a rich and glossy r...,beauty,nail polish,8.99,11.44,4.32,79,Nail Couture,...,None,None,https://cdn.dummyjson.com/product-images/beaut...,21.63,16.48,29.84,2025-04-30T09:41:02.053Z,2025-04-30T09:41:02.053Z,4063010628104,https://cdn.dummyjson.com/public/qr-code.png


In [11]:
column_mapping_products_transformed = {
    "id": "product_id",
    "discountPercentage": "discount_percentage",
    "warrantyInformation": "warranty_information",
    "shippingInformation": "shipping_information",
    "availabilityStatus": "availability_status",
    "returnPolicy": "return_policy",
    "minimumOrderQuantity": "min_order_quantity",
    "dimensions.width": "dimension_width",
    "dimensions.height": "dimension_height",
    "dimensions.depth": "dimension_depth",
    "meta.createdAt": "meta_created_at",
    "meta.updatedAt": "meta_updated_at",
    "meta.barcode": "meta_barcode",
    "meta.qrCode": "meta_qr_code"
}


load(df=products_df_transformed, table_name="products", schema="silver_prep", server = "DESKTOP-QBF1FTP\SQLEXPRESS", 
     database = "Dummydb", column_mapping = column_mapping_products_transformed)

✅ 194 rows loaded into silver_prep.products


In [12]:
carts_df_transformed = transform_carts(carts_df_raw, out_dir="processed_data")
carts_df_transformed.head()

,id,total,discountedTotal,userId,totalProducts,totalQuantity,product_id,product_title,product_price,product_quantity,product_total,product_discountPercentage,product_discountedTotal,product_thumbnail
0,1,103774.85,89686.65,33,4,15,168,Charger SXT RWD,32999.99,3,98999.97,13.39,85743.87,https://cdn.dummyjson.com/products/images/vehi...
1,1,103774.85,89686.65,33,4,15,78,Apple MacBook Pro 14 Inch Space Grey,1999.99,2,3999.98,18.52,3259.18,https://cdn.dummyjson.com/products/images/lapt...
2,1,103774.85,89686.65,33,4,15,183,Green Oval Earring,24.99,5,124.95,6.28,117.10,https://cdn.dummyjson.com/products/images/wome...
3,1,103774.85,89686.65,33,4,15,100,Apple Airpods,129.99,5,649.95,12.84,566.50,https://cdn.dummyjson.com/products/images/mobi...
4,2,4794.80,4288.95,142,5,20,144,Cricket Helmet,44.99,4,179.96,11.47,159.32,https://cdn.dummyjson.com/products/images/spor...


In [13]:
column_mapping_carts_transformed= {
    "id": "order_id",
    "discountedTotal": "discounted_total",
    "userId":"user_id",
    "totalProducts": "total_products",
    "totalQuantity": "total_quantity",
    "product_discountPercentage": "product_discount_percentage",
    "product_discountedTotal":"product_discounted_total"
}

load(df=carts_df_transformed, table_name="carts", schema="silver_prep", server = "DESKTOP-QBF1FTP\SQLEXPRESS", 
     database = "Dummydb", column_mapping = column_mapping_carts_transformed)

✅ 198 rows loaded into silver_prep.carts


In [14]:
users_df_transformed = users_df_raw
users_df_transformed.head()

,id,firstName,lastName,maidenName,age,gender,email,phone,username,password,...,company.address.city,company.address.state,company.address.stateCode,company.address.postalCode,company.address.coordinates.lat,company.address.coordinates.lng,company.address.country,crypto.coin,crypto.wallet,crypto.network
0,1,Emily,Johnson,Smith,28,female,emily.johnson@x.dummyjson.com,+81 965-431-3024,emilys,emilyspass,...,San Francisco,Wisconsin,WI,37657,71.814525,-161.150263,United States,Bitcoin,0xb9fc2fe63b2a6c003f1c324c3bfa53259162181a,Ethereum (ERC20)
1,2,Michael,Williams,,35,male,michael.williams@x.dummyjson.com,+49 258-627-6644,michaelw,michaelwpass,...,Los Angeles,New Hampshire,NH,73442,79.098326,-119.624845,United States,Bitcoin,0xb9fc2fe63b2a6c003f1c324c3bfa53259162181a,Ethereum (ERC20)
2,3,Sophia,Brown,,42,female,sophia.brown@x.dummyjson.com,+81 210-652-2785,sophiab,sophiabpass,...,Dallas,Nevada,NV,88511,20.086743,-34.577107,United States,Bitcoin,0xb9fc2fe63b2a6c003f1c324c3bfa53259162181a,Ethereum (ERC20)
3,4,James,Davis,,45,male,james.davis@x.dummyjson.com,+49 614-958-9364,jamesd,jamesdpass,...,Fort Worth,Pennsylvania,PA,27768,54.911930,-79.498328,United States,Bitcoin,0xb9fc2fe63b2a6c003f1c324c3bfa53259162181a,Ethereum (ERC20)
4,5,Emma,Miller,Johnson,30,female,emma.miller@x.dummyjson.com,+91 759-776-1614,emmaj,emmajpass,...,San Antonio,Idaho,ID,21965,44.346545,-26.944701,United States,Bitcoin,0xb9fc2fe63b2a6c003f1c324c3bfa53259162181a,Ethereum (ERC20)


In [15]:
column_mapping_users_transformed= {
    "id": "user_id",
    "firstName": "first_name",
    "lastName": "last_name",
    "maidenName": "maiden_name",
    "birthDate": "birthdate",
    "bloodGroup": "blood_group",
    "eyeColor": "eye_color",
    "macAddress": "mac_address",
    "userAgent": "user_agent",
    "hair.color": "hair_color",
    "hair.type": "hair_type",
    "address.address": "address_street",
    "address.city":"address_city",
    "address.state":"address_state",
    "address.stateCode": "address_state_code",
    "address.postalCode": "address_postal_code",
    "address.coordinates.lat": "address_coordinates_lat",
    "address.coordinates.lng": "address_coordinates_lng",
    "address.country":"address_country",
    "bank.cardExpire": "bank_card_expire",
    "bank.cardNumber": "bank_card_number",
    "bank.cardType": "bank_card_type",
    "bank.currency": "bank_currency",
    "bank.iban": "bank_iban",
    "company.department": "company_deparment",
    "company.name": "company_name",
    "company.title": "company_title",
    "company.address.address": "company_address_street",
    "company.address.city": "company_address_city",
    "company.address.state": "company_address_state",
    "company.address.stateCode": "company_address_state_code",
    "company.address.postalCode": "company_address_postal_code",
    "company.address.coordinates.lat": "company_address_coordinates_lat",
    "company.address.coordinates.lng": "company_address_coordinates_lng",
    "company.address.country": "company_address_country",
    "crypto.coin": "crypto_coin",
    "crypto.wallet": "crypto_wallet",
    "crypto.network": "crypto_network"
}


load(df=users_df_transformed, table_name="users", schema="silver_prep", server = "DESKTOP-QBF1FTP\SQLEXPRESS", 
     database = "Dummydb", column_mapping=column_mapping_users_transformed)

✅ 208 rows loaded into silver_prep.users


In [16]:
reviews_df = create_reviews_table(df=products_df_raw, out_dir="processed_data")
reviews_df

,review_id,product_id,rating,comment,date,reviewerName,reviewerEmail
0,1,1,3,Would not recommend!,2025-04-30T09:41:02.053Z,Eleanor Collins,eleanor.collins@x.dummyjson.com
1,2,1,4,Very satisfied!,2025-04-30T09:41:02.053Z,Lucas Gordon,lucas.gordon@x.dummyjson.com
2,3,1,5,Highly impressed!,2025-04-30T09:41:02.053Z,Eleanor Collins,eleanor.collins@x.dummyjson.com
3,4,2,5,Great product!,2025-04-30T09:41:02.053Z,Savannah Gomez,savannah.gomez@x.dummyjson.com
4,5,2,4,Awesome product!,2025-04-30T09:41:02.053Z,Christian Perez,christian.perez@x.dummyjson.com
...,...,...,...,...,...,...,...
577,578,193,4,Would buy again!,2025-04-30T09:41:02.054Z,Avery Barnes,avery.barnes@x.dummyjson.com
578,579,193,5,Great product!,2025-04-30T09:41:02.054Z,Evelyn Sanchez,evelyn.sanchez@x.dummyjson.com
579,580,194,4,Very happy with my purchase!,2025-04-30T09:41:02.054Z,Harper Kelly,harper.kelly@x.dummyjson.com
580,581,194,5,Highly recommended!,2025-04-30T09:41:02.054Z,Gabriel Bailey,gabriel.bailey@x.dummyjson.com


In [17]:
column_mapping_reviews = {"reviewerName": "reviewer_name",
                          "reviewerEmail":"reviewer_email"}

load(df=reviews_df, table_name='reviews', schema="silver_prep", server="DESKTOP-QBF1FTP\SQLEXPRESS", 
     database="Dummydb", column_mapping=column_mapping_reviews)

✅ 582 rows loaded into silver_prep.reviews
